In [194]:

import numpy as np

import pandas as pd


LIDAR_MAX_RANGE=20

In [195]:
'''
Read data from txt file using pandas
'''

df = pd.read_csv('scans.txt', delimiter = "\n")

#print(df)

            A
0    10.12000
1    10.15000
2    11.49500
3     9.21365
4     9.31250
..        ...
175  10.98400
176  10.35000
177  10.00000
178  10.25400
179  10.00000

[180 rows x 1 columns]


In [196]:
'''
Creating a Theta vector
'''
theta=np.arange(180)
#print(theta)

[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179]


In [197]:
'''
Storing the first column of the data fram to be the Lidar readings vector
'''
scanned_data = df.iloc[:, 0].values
print(scanned_data.shape)
#scanned_data=scanned_data.reshape(180,1)

(180,)


In [198]:
'''
Element wise multiplication of Theta and Lidar readings vectors in order to get the polar coordinates
Used numpy instead of making loops as numpy is fairly faster on bigger vectors, won't make big differnece on a 180 elemnt vector though
'''
cartesian= np.zeros(shape=(2, 180))

cartesian[0]= scanned_data*np.cos((theta*np.pi)/180)
cartesian[0]+=LIDAR_MAX_RANGE
cartesian[1]= scanned_data*np.sin((theta*np.pi)/180)
cartesian=np.floor(cartesian.T)
# j=0
# for i in range(0,180):
#     if(scanned_data[i]>0):
#         cartesian[j]=scanned_data[i]*np.cos((i*np.pi)/180),scanned_data[i]*np.sin((i*np.pi)/180)
#         j+=1

#cartesian=np.floor(cartesian)
#print(cartesian)


[[30.  0.]
 [30.  0.]
 [31.  0.]
 [29.  0.]
 [29.  0.]
 [30.  0.]
 [30.  1.]
 [29.  1.]
 [30.  1.]
 [29.  1.]
 [20.  0.]
 [20.  0.]
 [20.  0.]
 [20.  0.]
 [20.  0.]
 [20.  0.]
 [20.  0.]
 [20.  0.]
 [20.  0.]
 [20.  0.]
 [20.  0.]
 [20.  0.]
 [20.  0.]
 [20.  0.]
 [20.  0.]
 [20.  0.]
 [20.  0.]
 [20.  0.]
 [20.  0.]
 [20.  0.]
 [20.  0.]
 [20.  0.]
 [20.  0.]
 [20.  0.]
 [20.  0.]
 [20.  0.]
 [20.  0.]
 [20.  0.]
 [20.  0.]
 [20.  0.]
 [20.  0.]
 [20.  0.]
 [26.  6.]
 [27.  6.]
 [27.  7.]
 [28.  8.]
 [27.  7.]
 [20.  0.]
 [20.  0.]
 [20.  0.]
 [20.  0.]
 [20.  0.]
 [20.  0.]
 [20.  0.]
 [20.  0.]
 [20.  0.]
 [20.  0.]
 [20.  0.]
 [20.  0.]
 [20.  0.]
 [20.  0.]
 [20.  0.]
 [20.  0.]
 [20.  0.]
 [20.  0.]
 [20.  0.]
 [20.  0.]
 [20.  0.]
 [20.  0.]
 [20.  0.]
 [20.  0.]
 [20.  0.]
 [20.  0.]
 [20.  0.]
 [20.  0.]
 [20.  0.]
 [20.  0.]
 [20.  0.]
 [20.  0.]
 [20.  0.]
 [20.  0.]
 [20.  0.]
 [20.  0.]
 [20.  0.]
 [20.  0.]
 [20.  0.]
 [20.  0.]
 [20.  0.]
 [20. 15.]
 [20. 15.]
 [20. 15.]

In [200]:
'''
Creating the 2D grid and a 2d array 
'''

#grid is columns*rows 
#meaning that to index the grid it will be grid[y_coordinate][x_coordinate]
# grid shape is (height,width)=(lidar_max_range,2*lidar_max_range)
grid= np.zeros(shape=(LIDAR_MAX_RANGE,2*LIDAR_MAX_RANGE))


In [201]:
'''
This cell is to fill the grid with 1s in the place of objects seen by LIDAR
'''

for i in cartesian:
    x,y=i
    grid[int(y),int(x)]=1

binary_image=np.copy(grid)

In [202]:
def grid_index_exist(x,y):
    '''
    Function takes the x,y positoin of the grid and retruns whether postions is in bounds or not
    '''
    max_x,max_y=grid.shape
    if(x>=0 and x<max_x and y >=0 and y<max_y):
        return 1
    return 0

In [290]:
delta=np.array([[-1,-1],[-1, 0],[-1, 1],[ 0,-1],[ 0, 1], [ 1,-1],[ 1, 0],[ 1, 1]])

# for i in delta:
#     print(i[0]," ",i[1])

-1   -1
-1   0
-1   1
0   -1
0   1
1   -1
1   0
1   1


In [342]:
#visited array to be used as flags
visited= np.zeros(shape=(LIDAR_MAX_RANGE,2*LIDAR_MAX_RANGE))
# delta array is used to get the 8 neigghbours to a cell
delta=np.array([[-1,-1],[-1, 0],[-1, 1],[ 0,-1],[ 0, 1], [ 1,-1],[ 1, 0],[ 1, 1]])

#ressetting the grid to the binary_image in case the grid was labelled before
grid=np.copy(binary_image)

In [354]:

#x,y , x for row and y for column 
#unlike the real map having x for width and y for height 



def DFS (x,y,CC):
    '''
    takes the x,y position of the cell and check whether it has neighbours in order to mark them, using DFS

    Parameters
    ----------
    x : integer
        x postion of the cell 
    
    y: integer
        y positoin of the cell

    CC : integer
        the value to use inorder to mark the cells and its neighbours if exist any



    grid axees are as following   (rows,columns)=(x,y)
    -----------------------------------------y axis--------------------------------
    |
    |
    |
    |
    |
    x

    a
    x
    i
    s
    |
    |
    |
    |
    |
    '''
    visited[x][y]=1
    grid[x][y]=CC
    print("IN DFS")
    for neighbour in delta:
        new_x,new_y=x+neighbour[0],y+neighbour[1]
        if(grid_index_exist(new_x,new_y)):
            if(visited[new_x][new_y]==0 and grid[new_x][new_y]>0):
                print(new_x,' ',new_y)
                DFS(new_x,new_y,CC)
    
    return  


In [355]:
'''
Running DFS on the grid to number the connected components together
'''

delta=np.array([[-1,-1],[-1, 0],[-1, 1],[ 0,-1],[ 0, 1], [ 1,-1],[ 1, 0],[ 1, 1]])

CC=int(0)
print(grid.shape)
outter_loop,inner_loop=grid.shape

for i in range(0,outter_loop):
    for j in range (inner_loop):
        if(visited[i,j]==0 and grid[i,j]==1):
            CC+=1
            #CALL DFS and label the cells with CC meanwhile searching
            DFS(i,j,CC)
            pass
        pass


(20, 40)


In [356]:
# outter_loop,inner_loop=grid.shape
# print(outter_loop,"   ",inner_loop)
# print(visited.shape)
# print(visited)
# print("###################################################3")
# print(grid)

In [358]:
'''
This cell is to print the output
'''

output_file=open("out.txt","w")

for vector in grid:
    for element in vector:
        #temp=element.tostring()
        #print(str(element))
        if(element==0):
            output_file.write(".")
        else:
            output_file.write(str(int(element)))
        output_file.write(" ")
    output_file.write("\n")

output_file.close()
    